# Consensus Visualizer

Pyechart(Interactive) 차트를 사용하여 종목별 증권사 컨센서스 현황을 시각화

In [2]:
import pandas as pd
import numpy as np
import pyecharts
print("pyecharts version : ", pyecharts.__version__)
import datetime
import sqlite3
from jupyterthemes import jtplot
jtplot.style('grade3')

pyecharts version :  0.5.10


In [27]:
con=sqlite3.connect('C:\\Users\\wnsvy\\PycharmProjects\\Crawling_Practice\\consensus.db')
cursor=con.cursor()

In [28]:
sql_state = "SELECT * FROM " + "business" +" ORDER BY dt DESC LIMIT 1"
df = pd.read_sql(sql_state, con, index_col='dt')
recent_date = df.index[0]
recent_date = datetime.datetime.strptime(recent_date, '%Y-%m-%d')
recent_date

datetime.datetime(2018, 10, 25, 0, 0)

### 점검일 설정

#### start_date -> recent_date

In [29]:
# 테스트용
recent_date = '2018-06-27'
recent_date = datetime.datetime.strptime(recent_date, '%Y-%m-%d')
recent_date

datetime.datetime(2018, 6, 27, 0, 0)

In [30]:
# 기간 1년 default
time_period = datetime.timedelta(days=365)
time_period

datetime.timedelta(365)

In [31]:
start_date = recent_date - time_period
start_date

datetime.datetime(2017, 6, 27, 0, 0)

In [32]:
datetime.datetime.strftime(start_date, '%Y-%m-%d')

'2017-06-27'

## SQL 쿼리하여 Consensus 데이터 읽기

In [33]:
sql_state = "SELECT * FROM " + "stock_good_price" +" WHERE dt > " + "'" + datetime.datetime.strftime(start_date, '%Y-%m-%d') + "'"
print(sql_state)
up_price = pd.read_sql(sql_state, con, index_col='dt')
up_price.index = pd.Series(up_price.index).apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))
up_price = up_price[up_price.index <= recent_date]

SELECT * FROM stock_good_price WHERE dt > '2017-06-27'


In [34]:
sql_state = "SELECT * FROM " + "stock_bad_price" +" WHERE dt > " + "'" + datetime.datetime.strftime(start_date, '%Y-%m-%d') + "'"
print(sql_state)
down_price = pd.read_sql(sql_state, con, index_col='dt')
down_price.index = pd.Series(down_price.index).apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))
down_price = down_price[down_price.index <= recent_date]
down_price.head()

SELECT * FROM stock_bad_price WHERE dt > '2017-06-27'


,title,writer,issued_by,current,prev,code,company
dt,,,,,,,
2017-06-28,에스에프에이(056190)17년 2분기 Preview 에스에프에이(056190)17...,이상언,유안타증권,69000,110000,A056190,에스에프에이
2017-06-28,에스에프에이(056190)안정적인 실적을 자랑하는 장비 업체 에스에프에이(05619...,"윤영식,소현철",신한금융투자,54000,105000,A056190,에스에프에이
2017-06-28,인터파크(108790)투어 부문의 비상을 기다리는 시간 인터파크(108790)투어 ...,이효진,NH투자증권,13000,15000,A108790,인터파크
2017-06-29,대한유화(006650)지나간 2분기 vs. 다가올 하반기 대한유화(006650)지나...,"한상원,이응주",신한금융투자,320000,340000,A006650,대한유화
2017-06-29,코스맥스(192820)업황 부진과 투자 확대의 이중부담 지속 코스맥스(192820)...,"양지혜,윤보미",메리츠종금증권,140000,170000,A192820,코스맥스


In [35]:
up_price.tail()

,title,writer,issued_by,current,prev,code,company
dt,,,,,,,
2018-06-26,고영(098460)AOI 모멘텀 가속 고영(098460)AOI 모멘텀 가속 AOI ...,"도현우,서준현",NH투자증권,130000,120000,A098460,고영
2018-06-27,신세계인터내셔날(031430)성장의 서막 신세계인터내셔날(031430)성장의 서막 ...,하누리,미래에셋대우,260000,200000,A031430,신세계인터내셔날
2018-06-27,한국콜마(161890)바닥 잡기 한국콜마(161890)바닥 잡기 한국콜마(16189...,"이선화,장소라",유진투자증권,105000,83500,A161890,한국콜마
2018-06-27,휠라코리아(081660)Rebuilding brand value “AGAIN” 휠라...,유정현,대신증권,43000,36000,A081660,휠라코리아
2018-06-27,한국콜마(161890)2Q18 프리뷰: 국내 화장품 매출액 40% 이상 성... 한...,"박신애,송재원",KB증권,95000,92000,A161890,한국콜마


In [36]:
up_price['current'].tail()

dt
2018-06-26    130000
2018-06-27    260000
2018-06-27    105000
2018-06-27     43000
2018-06-27     95000
Name: current, dtype: int64

## 해당 종목에 대한 가격정보 읽기

antQuant.utils.DataManager 모듈 사용

In [37]:
from antQuant.utils import DataManager

In [51]:
# code = '067160' # 아프리카TV
# code = '000660' # SK Hynix
# code = '051910' # LG Chem
# code = '035900' # JYP
code= '207940' # 삼성바이오로직스
# code = '011170' # 롯데 케미칼

In [52]:
dm = DataManager()
df = dm.get_adj_daily_ohlcv(code)

In [53]:
df.tail()

,open,high,low,close,volume
date,,,,,
2018-06-21,420000.0,426000.0,411000.0,415000.0,153582.0
2018-06-22,419500.0,445500.0,415500.0,437500.0,345522.0
2018-06-25,438000.0,438500.0,418000.0,420500.0,165310.0
2018-06-26,410000.0,419000.0,400500.0,416000.0,217564.0
2018-06-27,414500.0,426500.0,413000.0,418000.0,116211.0


In [54]:
df.index[0]

Timestamp('2016-11-10 00:00:00')

## 해당 종목에 해당하는 Opinion 추출

In [55]:
price = df[df.index > start_date]

In [56]:
# price = price['close']
price = price['high']
idx = price.index

In [57]:
selected_down_data = down_price[down_price['code']=='A' + code]

In [58]:
selected_up_data = up_price[up_price['code']=='A' + code]
selected_up_data.head()

,title,writer,issued_by,current,prev,code,company
dt,,,,,,,
2017-07-25,"삼성바이오로직스(207940)삼성 바이오 사업 예정대로 순항, 목표... 삼성바이오...","이승호,박원용",삼성증권,340000,230000,A207940,삼성바이오로직스
2017-09-18,삼성바이오로직스(207940)바이오시밀러 두번째 first-mover 장착 삼성바이...,진홍국,한국투자증권,400000,350000,A207940,삼성바이오로직스
2017-10-13,삼성바이오로직스(207940)바이오시밀러는 생산공장도 중요하다! 삼성바이오로직스(2...,진홍국,한국투자증권,440000,400000,A207940,삼성바이오로직스
2017-10-26,삼성바이오로직스(207940)예상보다 빠른 바이오 CMO 손익분기점 ... 삼성바이...,이승호,삼성증권,420000,340000,A207940,삼성바이오로직스
2017-10-26,삼성바이오로직스(207940)예정된 턴어라운드 삼성바이오로직스(207940)예정된 ...,홍가혜,대신증권,410000,300000,A207940,삼성바이오로직스


In [59]:
current = selected_up_data['current']
prev = selected_up_data['prev']

In [60]:
down_current = selected_down_data['current']
down_prev = selected_down_data['prev']

In [61]:
price.min()

264500.0

## 결과 Plot

In [62]:
from pyecharts import Scatter, Line, Overlap, EffectScatter

line = Line("Opinion Visual")
line.add("Close",idx,price,xaxis_type='time', symbol=None,is_datazoom_show=True,datazoom_type='both'
        ,is_yaxislabel_align=True,yaxis_min=price.min() * 0.9)

# scatter : 일반 인디케이터 표시
# effect scatter : 애니메이션 있는 인디케이터 표시

scatter = Scatter()
ef_scatter = EffectScatter()

ef_scatter.add("up_goal",current.index,current,symbol_size=6, effect_scale=2.5, effect_period=3,symbol="triangle")
scatter.add("up_previous",prev.index,prev,xaxis_type='time',symbol_size=8, symbol='triangle')

ef_scatter.add("down_goal",down_current.index,down_current,symbol_size=5, effect_scale=3.5, effect_brushtype='fill',symbol="diamond")
scatter.add("down_previous",down_prev.index,down_prev,xaxis_type='time',symbol_size=8, symbol='diamond')

#ef_scatter.add("current",current.index,current,symbol_size=5, effect_scale=6.5, effect_brushtype='fill',symbol="diamond")
#ef_scatter.add("prev",prev.index,prev,symbol_size=6, effect_scale=2.5, effect_period=3,symbol="triangle")

overlap = Overlap("Upper Prediction")
overlap.add(line)

overlap.add(scatter)
overlap.add(ef_scatter)

# overlap.render(is_datazoom_show=True)
overlap.width = 1000
overlap.height = 500
overlap